In [1]:
import csv

In [67]:
export_file = '/Users/jdamerow/UpConsulting/files/latex/2019-10-08 23_43_48--export-for-latex-2.csv'
result_file_path = '/Users/jdamerow/UpConsulting/files/latex/result.txt'

In [91]:
class Author:
    def __init__(self):
        self.name = ''
        self.order = 0

In [92]:
class Book:
    def __init__(self):
        self.authors = []
        self.title = ''
        self.year = 0
        self.publisher = ''
        self.isbn = ''
        self.isiscb_id = ''

In [93]:
def create_author(author_string):
    author = Author()
    parts = author_string.split('||')
    for part in parts:
        part = part.strip()
        if part.startswith('AuthorityName '):
            author.name = part[len('AuthorityName '):]
            continue
        if part.startswith('ACRDisplayOrder '):
            author.order = part[len('ACRDisplayOrder '):]
    return author

In [94]:
def get_publisher(publisher_string):
    parts = publisher_string.split('||')
    for part in parts:
        part = part.strip()
        if part.startswith('AuthorityName '):
            return part[len('AuthorityName '):]

In [95]:
def build_book(row):
    book = Book()
    book.isiscb_id = row['Record ID']
    
    author_strings = row['Author'].split('//')
    for a in author_strings:
        book.authors.append(create_author(a))
    
    
    book.title = row['Title']
    book.year = row['Year of publication']
    book.publisher = get_publisher(row['Place Publisher'])
    book.isbn = row['ISBN']
    return book

In [96]:
latex_template = """
\\noindent\\begin{footnotesize}\\textbf{%s}\\hspace{0.5em} %s. \\textit{%s}. %s, %s. %s \\textsc{\\href{https://data.isiscb.org/isis/citation/%s}{%s}}.
\\vspace{2ex} \\end{footnotesize}
\\paragraph{}
\n
"""
author_template = "\\textsc{%s}, %s"
isbn_template = "\\textsc{isbn}: %s."
with open(export_file) as csvfile:
    with open(result_file_path, 'w') as result_file:
        reader = csv.DictReader(csvfile)
        counter = 1
        for row in reader:
            if row['Record Type'] != "Book":
                continue
            book = build_book(row)
            authors_latex = []
            for a in book.authors:
                name = a.name
                if ',' in name:
                    last = name[:name.index(',')]
                    first = name[name.index(',')+1:]
                else:
                    last = name[name.rfind(" "):].strip()
                    first = name[:name.rfind(" ")].strip()
                authors_latex.append(author_template%(last, first))
            
            isbn = ''
            if book.isbn:
                isbn = isbn_template%(book.isbn)
            latex_entry = latex_template%(counter, ", ".join(authors_latex), book.title, book.publisher, book.year, isbn, book.isiscb_id, book.isiscb_id)

            result_file.write(latex_entry.strip())
            counter += 1
